In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import warnings
warnings.simplefilter(action='ignore')

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [3]:
class neuroverkko(nn.Module):
    def __init__(self):
        super(neuroverkko, self).__init__()
        self.hidden = nn.Linear(4, 10)  # Piilotettu kerros, jossa 10 neuronia
        self.output = nn.Linear(10, 3)  # Ulostulokerros, jossa 3 neuronia

    def forward(self, x):
        x = torch.relu(self.hidden(x))  # Aktivointifunktio piilotetussa kerroksessa
        x = self.output(x)  # Ulostulo ilman softmaxia (CrossEntropyLoss hoitaa tämän)
        return x

model = neuroverkko()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
for epoch in range(50):
    # Koulutustarkkuus
    model.train()
    correct_train = 0
    total_train = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        # Laske ennusteet ja tarkkuus koulutusaineistolla
        _, predicted_train = torch.max(outputs, 1)
        total_train += batch_y.size(0)
        correct_train += (predicted_train == batch_y).sum().item()
    train_accuracy = 100 * correct_train / total_train

    # Validointitarkkuus
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            val_outputs = model(batch_X)
            _, predicted_val = torch.max(val_outputs, 1)
            total_val += batch_y.size(0)
            correct_val += (predicted_val == batch_y).sum().item()
    val_accuracy = 100 * correct_val / total_val

    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1, Loss: 0.0231, Train Accuracy: 96.67%, Validation Accuracy: 96.67%
Epoch 2, Loss: 0.0094, Train Accuracy: 96.67%, Validation Accuracy: 96.67%
Epoch 3, Loss: 0.1046, Train Accuracy: 97.50%, Validation Accuracy: 93.33%
Epoch 4, Loss: 0.0875, Train Accuracy: 98.33%, Validation Accuracy: 96.67%
Epoch 5, Loss: 0.0012, Train Accuracy: 97.50%, Validation Accuracy: 96.67%
Epoch 6, Loss: 0.0203, Train Accuracy: 96.67%, Validation Accuracy: 100.00%
Epoch 7, Loss: 0.0069, Train Accuracy: 97.50%, Validation Accuracy: 100.00%
Epoch 8, Loss: 0.0021, Train Accuracy: 96.67%, Validation Accuracy: 96.67%
Epoch 9, Loss: 0.1393, Train Accuracy: 97.50%, Validation Accuracy: 100.00%
Epoch 10, Loss: 0.0073, Train Accuracy: 97.50%, Validation Accuracy: 100.00%
Epoch 11, Loss: 0.1751, Train Accuracy: 97.50%, Validation Accuracy: 96.67%
Epoch 12, Loss: 0.0013, Train Accuracy: 97.50%, Validation Accuracy: 93.33%
Epoch 13, Loss: 0.1130, Train Accuracy: 95.83%, Validation Accuracy: 100.00%
Epoch 14, Loss: 